In [8]:
from marketing_attribution_models import MAM
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
from markovclick.models import MarkovClickstream
from markovclick.viz import visualise_markov_chain
import os
import graphviz
import matplotlib as mpl
from pandas.io import gbq
import pandas_gbq
import glob
from pylab import *
import tempfile
import json
from datetime import timedelta
import seaborn as sns
import gc
from datetime import datetime
import re
from google.cloud import bigquery

################################################# Data Loading  #########################################

project = "ft-customer-analytics"
location = "EU"
client = bigquery.Client(project=project, location=location)

################################################# Define variables #################################################

ids = "user_guid"
date = "attribution_visit_start_time"
touchpoint = "touchpoint"
transaction = "converting_visit"

################################################# Define the date range for processing #################################################

end_date = pd.Timestamp.today().date() - pd.DateOffset(days=1)
start_date = end_date - pd.DateOffset(days=14)

start_date = start_date.date()
end_date = end_date.date() 

table_id = "ft-customer-analytics.crg_nniu.stg_conversion_users_last_15_days_90_days_lookback_table"

################################################# Output DataFrames  #################################################


In [9]:
################################################# Output DataFrames  #################################################

attribution_df_all_regis_90 = pd.DataFrame()
normalized_removal_effects_all_regis_90 = pd.DataFrame()
markov_transition_matrix_all_regis_90 = pd.DataFrame()
user_df_all_regis_90 = pd.DataFrame()
conversion_window_df_regis = pd.DataFrame()

################################################# Process Data for Each Day #########################################

for current_date in pd.date_range(start_date, end_date, freq="D"):
    # Create SQL query for the current date
    query = f"""
    SELECT * FROM {table_id}
    WHERE DATE(conversion_visit_timestamp) = "{current_date.strftime('%Y-%m-%d')}"
    """
    print(f"Fetching data for {current_date.strftime('%Y-%m-%d')}")


    # Execute the query
    query_job = client.query(query)
    df = query_job.to_dataframe()

    if df.empty:
        print(f"No data for {current_date.strftime('%Y-%m-%d')}")
        continue

    ################################################# Data Cleaning  #########################################
    
    df["original_transaction"] = df["converting_visit"]
    regis_df = df[df["conversion_type"] == "registration"].drop(columns=["conversion_type"])

    regis_df["user_max_date"] = regis_df.groupby(ids)["conversion_visit_timestamp"].transform("max")
    regis_df[transaction] = 0
    regis_df.loc[(regis_df[date] == regis_df["user_max_date"]) & (regis_df["original_transaction"] == 1), transaction] = 1

    regis_df = regis_df.sort_values([ids, date], ascending=[False, True])

    regis_df["run_date"] = current_date.date()

    ################################################# Median day calculation #########################################
    
    # Initialize a list to store each user's median time to register
    user_median_days = []

    # Calculate the median days for each user
    for user_guid, user_data in regis_df.groupby(ids):
        # Find the earliest visit where transaction = 0 (initial visit)
        first_visit = user_data[user_data[transaction] == 0][date].min()

        # If no valid first visit is found, skip this user
        if pd.isnull(first_visit):
            continue

        # Find the conversion date (transaction = 1)
        conversion_date = user_data[user_data[transaction] == 1][date].min()

        # Calculate the time difference in days
        if pd.notnull(conversion_date):
            days_to_convert = (conversion_date - first_visit).days
            user_median_days.append(days_to_convert)

    # Calculate the median of the user's conversion times
    if user_median_days:
        median_days_to_register = pd.Series(user_median_days).median()
    else:
        median_days_to_register = None  # If no data, set median as None

    # Add the calculated median days and run date to the output DataFrame
    conversion_window_df_regis = pd.concat(
        [
            conversion_window_df_regis,
            pd.DataFrame(
                {
                    "stage": ["registration"],
                    "median_days": [median_days_to_register],
                    "run_date": [current_date.date()],
                }
            ),
        ],
        ignore_index=True,
    )


    ################################################# MAM Initialization #########################################

    try:
        # Initialize the MAM class
        attributions = MAM(
            regis_df,
            group_channels=True,
            channels_colname=touchpoint,
            journey_with_conv_colname=transaction,
            group_channels_by_id_list=[ids],
            group_timestamp_colname=date,
            create_journey_id_based_on_conversion=True,
        )

        ################################################# Apply Attribution Models #########################################

        attributions.attribution_last_click()
        attributions.attribution_first_click()
        attributions.attribution_position_based(
            list_positions_first_middle_last=[0.3, 0.3, 0.4]
        )
        attributions.attribution_time_decay(
            decay_over_time=0.6, frequency=7
        )  # Frequency in hours
        attribution_markov = attributions.attribution_markov(
            transition_to_same_state=False
        )

        ################################################# Process Results #########################################

        # User-level attribution data
        user_df_temp = attributions.as_pd_dataframe()
        user_df_temp["num_touchpoints"] = (
            user_df_temp["channels_agg"].str.split(" > ").apply(len)
        )
        user_df_temp["run_date"] = current_date.date()

        # Extract user_guid from journey_id
        user_df_temp['user_guid'] = user_df_temp['journey_id'].str.extract(r'id:(.*)_J:0')[0]

        # Prepare df for merging
        df['conversion_visit_timestamp_date'] = df['conversion_visit_timestamp'].dt.date
        product_arrangement_df = df[['user_guid', 'conversion_visit_timestamp_date', 'product_arrangement_id']].drop_duplicates()

        # Merge user_df_temp with product_arrangement_df
        user_df_temp = user_df_temp.merge(
            product_arrangement_df,
            left_on=['user_guid', 'run_date'],
            right_on=['user_guid', 'conversion_visit_timestamp_date'],
            how='left'
        )

        # Drop 'conversion_visit_timestamp_date' column after merge
        user_df_temp.drop(columns=['conversion_visit_timestamp_date'], inplace=True)

        # Now concatenate user_df_temp into user_df_all_regis_90
        user_df_all_regis_90 = pd.concat(
            [user_df_all_regis_90, user_df_temp], ignore_index=True
        )

        # Proceed with processing markov_transition_matrix and other dataframes as before
        # Markov transition matrix
        markov_transition_matrix = attribution_markov[2].round(3)
        markov_transition_matrix = markov_transition_matrix.rename(
            index=lambda x: x.replace("(inicio)", "(start)"),
            columns=lambda x: x.replace("(inicio)", "(start)"),
        )
        markov_transition_matrix.reset_index(inplace=True)
        markov_transition_matrix = pd.melt(
            markov_transition_matrix,
            id_vars="index",
            var_name="destination",
            value_name="probability",
        )
        markov_transition_matrix.columns = ["source", "destination", "probability"]
        markov_transition_matrix["run_date"] = current_date.date()
        markov_transition_matrix_all_regis_90 = pd.concat(
            [markov_transition_matrix_all_regis_90, markov_transition_matrix],
            ignore_index=True,
        )

        # Removal effects
        removal_effect_matrix = attribution_markov[3].round(3)
        channels = removal_effect_matrix.index
        removal_effect_values = removal_effect_matrix[["removal_effect"]]
        normalized_values = (removal_effect_values / removal_effect_values.sum()) * 100
        normalized_removal_effects = pd.DataFrame(
            normalized_values, index=channels, columns=["removal_effect"]
        )
        normalized_removal_effects["run_date"] = current_date.date()
        normalized_removal_effects["removal_effect_raw"] = (
            removal_effect_values.values.flatten()
        )
        normalized_removal_effects.reset_index(inplace=True)
        normalized_removal_effects.rename(columns={"index": "channel"}, inplace=True)
        normalized_removal_effects_all_regis_90 = pd.concat(
            [normalized_removal_effects_all_regis_90, normalized_removal_effects],
            ignore_index=True,
        )

        # Attribution by channels and models
        attribution_df = attributions.group_by_channels_models
        attribution_df["run_date"] = current_date.date()
        attribution_df.columns = attribution_df.columns.str.replace(
            ".", "_", regex=False
        ).str.replace(" ", "_", regex=False)
        attribution_df_all_regis_90 = pd.concat(
            [attribution_df_all_regis_90, attribution_df], ignore_index=True
        )

        print(f"Processed data for {current_date.strftime('%Y-%m-%d')}")

    except Exception as e:
        print(
            f"An error occurred for the date {current_date.strftime('%Y-%m-%d')}: {e}"
        )
        continue

################################################# Finalize Results #########################################

attribution_df_all_regis_90["conversion_window"] = 90
normalized_removal_effects_all_regis_90["conversion_window"] = 90
markov_transition_matrix_all_regis_90["conversion_window"] = 90
user_df_all_regis_90["conversion_window"] = 90

attribution_df_all_regis_90["conversion_type"] = "registration"
normalized_removal_effects_all_regis_90["conversion_type"] = "registration"
markov_transition_matrix_all_regis_90["conversion_type"] = "registration"
user_df_all_regis_90["conversion_type"] = "registration"

Fetching data for 2025-02-19


/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


Processed data for 2025-02-19
Fetching data for 2025-02-20


/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


Processed data for 2025-02-20
Fetching data for 2025-02-21


...

Processed data for 2025-03-03
Fetching data for 2025-03-04


/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


An error occurred for the date 2025-03-04: cumsum is not supported for object dtype
Fetching data for 2025-03-05


/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


An error occurred for the date 2025-03-05: cumsum is not supported for object dtype


In [ ]:
################################################# Output DataFrames  #################################################
table_id = "ft-customer-analytics.crg_nniu.stg_conversion_users_last_15_days_60_days_lookback_table"

attribution_df_all_regis_60 = pd.DataFrame()
normalized_removal_effects_all_regis_60 = pd.DataFrame()
markov_transition_matrix_all_regis_60 = pd.DataFrame()
user_df_all_regis_60 = pd.DataFrame()
conversion_window_df_regis = pd.DataFrame()

################################################# Process Data for Each Day #########################################

for current_date in pd.date_range(start_date, end_date, freq="D"):
    # Create SQL query for the current date
    query = f"""
    SELECT * FROM {table_id}
    WHERE DATE(conversion_visit_timestamp) = "{current_date.strftime('%Y-%m-%d')}"
    """
    print(f"Fetching data for {current_date.strftime('%Y-%m-%d')}")


    # Execute the query
    query_job = client.query(query)
    df = query_job.to_dataframe()

    if df.empty:
        print(f"No data for {current_date.strftime('%Y-%m-%d')}")
        continue

    ################################################# Data Cleaning  #########################################
    
    df["original_transaction"] = df["converting_visit"]
    regis_df = df[df["conversion_type"] == "registration"].drop(columns=["conversion_type"])

    regis_df["user_max_date"] = regis_df.groupby(ids)["conversion_visit_timestamp"].transform("max")
    regis_df[transaction] = 0
    regis_df.loc[(regis_df[date] == regis_df["user_max_date"]) & (regis_df["original_transaction"] == 1), transaction] = 1

    regis_df = regis_df.sort_values([ids, date], ascending=[False, True])

    regis_df["run_date"] = current_date.date()

    ################################################# Median day calculation #########################################
    
    # Initialize a list to store each user's median time to register
    user_median_days = []

    # Calculate the median days for each user
    for user_guid, user_data in regis_df.groupby(ids):
        # Find the earliest visit where transaction = 0 (initial visit)
        first_visit = user_data[user_data[transaction] == 0][date].min()

        # If no valid first visit is found, skip this user
        if pd.isnull(first_visit):
            continue

        # Find the conversion date (transaction = 1)
        conversion_date = user_data[user_data[transaction] == 1][date].min()

        # Calculate the time difference in days
        if pd.notnull(conversion_date):
            days_to_convert = (conversion_date - first_visit).days
            user_median_days.append(days_to_convert)

    # Calculate the median of the user's conversion times
    if user_median_days:
        median_days_to_register = pd.Series(user_median_days).median()
    else:
        median_days_to_register = None  # If no data, set median as None

    # Add the calculated median days and run date to the output DataFrame
    conversion_window_df_regis = pd.concat(
        [
            conversion_window_df_regis,
            pd.DataFrame(
                {
                    "stage": ["registration"],
                    "median_days": [median_days_to_register],
                    "run_date": [current_date.date()],
                }
            ),
        ],
        ignore_index=True,
    )


    ################################################# MAM Initialization #########################################

    try:
        # Initialize the MAM class
        attributions = MAM(
            regis_df,
            group_channels=True,
            channels_colname=touchpoint,
            journey_with_conv_colname=transaction,
            group_channels_by_id_list=[ids],
            group_timestamp_colname=date,
            create_journey_id_based_on_conversion=True,
        )

        ################################################# Apply Attribution Models #########################################

        attributions.attribution_last_click()
        attributions.attribution_first_click()
        attributions.attribution_position_based(
            list_positions_first_middle_last=[0.3, 0.3, 0.4]
        )
        attributions.attribution_time_decay(
            decay_over_time=0.6, frequency=7
        )  # Frequency in hours
        attribution_markov = attributions.attribution_markov(
            transition_to_same_state=False
        )

        ################################################# Process Results #########################################

        # User-level attribution data
        user_df_temp = attributions.as_pd_dataframe()
        user_df_temp["num_touchpoints"] = (
            user_df_temp["channels_agg"].str.split(" > ").apply(len)
        )
        user_df_temp["run_date"] = current_date.date()

        # Extract user_guid from journey_id
        user_df_temp['user_guid'] = user_df_temp['journey_id'].str.extract(r'id:(.*)_J:0')[0]

        # Prepare df for merging
        df['conversion_visit_timestamp_date'] = df['conversion_visit_timestamp'].dt.date
        product_arrangement_df = df[['user_guid', 'conversion_visit_timestamp_date', 'product_arrangement_id']].drop_duplicates()

        # Merge user_df_temp with product_arrangement_df
        user_df_temp = user_df_temp.merge(
            product_arrangement_df,
            left_on=['user_guid', 'run_date'],
            right_on=['user_guid', 'conversion_visit_timestamp_date'],
            how='left'
        )

        # Drop 'conversion_visit_timestamp_date' column after merge
        user_df_temp.drop(columns=['conversion_visit_timestamp_date'], inplace=True)

        # Now concatenate user_df_temp into user_df_all_regis_60
        user_df_all_regis_60 = pd.concat(
            [user_df_all_regis_60, user_df_temp], ignore_index=True
        )

        # Proceed with processing markov_transition_matrix and other dataframes as before
        # Markov transition matrix
        markov_transition_matrix = attribution_markov[2].round(3)
        markov_transition_matrix = markov_transition_matrix.rename(
            index=lambda x: x.replace("(inicio)", "(start)"),
            columns=lambda x: x.replace("(inicio)", "(start)"),
        )
        markov_transition_matrix.reset_index(inplace=True)
        markov_transition_matrix = pd.melt(
            markov_transition_matrix,
            id_vars="index",
            var_name="destination",
            value_name="probability",
        )
        markov_transition_matrix.columns = ["source", "destination", "probability"]
        markov_transition_matrix["run_date"] = current_date.date()
        markov_transition_matrix_all_regis_60 = pd.concat(
            [markov_transition_matrix_all_regis_60, markov_transition_matrix],
            ignore_index=True,
        )

        # Removal effects
        removal_effect_matrix = attribution_markov[3].round(3)
        channels = removal_effect_matrix.index
        removal_effect_values = removal_effect_matrix[["removal_effect"]]
        normalized_values = (removal_effect_values / removal_effect_values.sum()) * 100
        normalized_removal_effects = pd.DataFrame(
            normalized_values, index=channels, columns=["removal_effect"]
        )
        normalized_removal_effects["run_date"] = current_date.date()
        normalized_removal_effects["removal_effect_raw"] = (
            removal_effect_values.values.flatten()
        )
        normalized_removal_effects.reset_index(inplace=True)
        normalized_removal_effects.rename(columns={"index": "channel"}, inplace=True)
        normalized_removal_effects_all_regis_60 = pd.concat(
            [normalized_removal_effects_all_regis_60, normalized_removal_effects],
            ignore_index=True,
        )

        # Attribution by channels and models
        attribution_df = attributions.group_by_channels_models
        attribution_df["run_date"] = current_date.date()
        attribution_df.columns = attribution_df.columns.str.replace(
            ".", "_", regex=False
        ).str.replace(" ", "_", regex=False)
        attribution_df_all_regis_60 = pd.concat(
            [attribution_df_all_regis_60, attribution_df], ignore_index=True
        )

        print(f"Processed data for {current_date.strftime('%Y-%m-%d')}")

    except Exception as e:
        print(
            f"An error occurred for the date {current_date.strftime('%Y-%m-%d')}: {e}"
        )
        continue

################################################# Finalize Results #########################################

attribution_df_all_regis_60["conversion_window"] = 60
normalized_removal_effects_all_regis_60["conversion_window"] = 60
markov_transition_matrix_all_regis_60["conversion_window"] = 60
user_df_all_regis_60["conversion_window"] = 60

attribution_df_all_regis_60["conversion_type"] = "registration"
normalized_removal_effects_all_regis_60["conversion_type"] = "registration"
markov_transition_matrix_all_regis_60["conversion_type"] = "registration"
user_df_all_regis_60["conversion_type"] = "registration"

Fetching data for 2025-02-16


/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


Processed data for 2025-02-16
Fetching data for 2025-02-17


/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


Processed data for 2025-02-17
Fetching data for 2025-02-18


...

An error occurred for the date 2025-02-28: cumsum is not supported for object dtype
Fetching data for 2025-03-01


/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


An error occurred for the date 2025-03-01: cumsum is not supported for object dtype
Fetching data for 2025-03-02


/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/anaconda3/envs/clean_env/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


An error occurred for the date 2025-03-02: cumsum is not supported for object dtype


In [ ]:
################################################# Output DataFrames  #################################################
table_id = "ft-customer-analytics.crg_nniu.stg_conversion_users_last_15_days_30_days_lookback_table"

attribution_df_all_regis_30 = pd.DataFrame()
normalized_removal_effects_all_regis_30 = pd.DataFrame()
markov_transition_matrix_all_regis_30 = pd.DataFrame()
user_df_all_regis_30 = pd.DataFrame()
conversion_window_df_regis = pd.DataFrame()

################################################# Process Data for Each Day #########################################

for current_date in pd.date_range(start_date, end_date, freq="D"):
    # Create SQL query for the current date
    query = f"""
    SELECT * FROM {table_id}
    WHERE DATE(conversion_visit_timestamp) = "{current_date.strftime('%Y-%m-%d')}"
    """
    print(f"Fetching data for {current_date.strftime('%Y-%m-%d')}")


    # Execute the query
    query_job = client.query(query)
    df = query_job.to_dataframe()

    if df.empty:
        print(f"No data for {current_date.strftime('%Y-%m-%d')}")
        continue

    ################################################# Data Cleaning  #########################################
    
    df["original_transaction"] = df["converting_visit"]
    regis_df = df[df["conversion_type"] == "registration"].drop(columns=["conversion_type"])

    regis_df["user_max_date"] = regis_df.groupby(ids)["conversion_visit_timestamp"].transform("max")
    regis_df[transaction] = 0
    regis_df.loc[(regis_df[date] == regis_df["user_max_date"]) & (regis_df["original_transaction"] == 1), transaction] = 1

    regis_df = regis_df.sort_values([ids, date], ascending=[False, True])

    regis_df["run_date"] = current_date.date()

    ################################################# Median day calculation #########################################
    
    # Initialize a list to store each user's median time to register
    user_median_days = []

    # Calculate the median days for each user
    for user_guid, user_data in regis_df.groupby(ids):
        # Find the earliest visit where transaction = 0 (initial visit)
        first_visit = user_data[user_data[transaction] == 0][date].min()

        # If no valid first visit is found, skip this user
        if pd.isnull(first_visit):
            continue

        # Find the conversion date (transaction = 1)
        conversion_date = user_data[user_data[transaction] == 1][date].min()

        # Calculate the time difference in days
        if pd.notnull(conversion_date):
            days_to_convert = (conversion_date - first_visit).days
            user_median_days.append(days_to_convert)

    # Calculate the median of the user's conversion times
    if user_median_days:
        median_days_to_register = pd.Series(user_median_days).median()
    else:
        median_days_to_register = None  # If no data, set median as None

    # Add the calculated median days and run date to the output DataFrame
    conversion_window_df_regis = pd.concat(
        [
            conversion_window_df_regis,
            pd.DataFrame(
                {
                    "stage": ["registration"],
                    "median_days": [median_days_to_register],
                    "run_date": [current_date.date()],
                }
            ),
        ],
        ignore_index=True,
    )


    ################################################# MAM Initialization #########################################

    try:
        # Initialize the MAM class
        attributions = MAM(
            regis_df,
            group_channels=True,
            channels_colname=touchpoint,
            journey_with_conv_colname=transaction,
            group_channels_by_id_list=[ids],
            group_timestamp_colname=date,
            create_journey_id_based_on_conversion=True,
        )

        ################################################# Apply Attribution Models #########################################

        attributions.attribution_last_click()
        attributions.attribution_first_click()
        attributions.attribution_position_based(
            list_positions_first_middle_last=[0.3, 0.3, 0.4]
        )
        attributions.attribution_time_decay(
            decay_over_time=0.6, frequency=7
        )  # Frequency in hours
        attribution_markov = attributions.attribution_markov(
            transition_to_same_state=False
        )

        ################################################# Process Results #########################################

        # User-level attribution data
        user_df_temp = attributions.as_pd_dataframe()
        user_df_temp["num_touchpoints"] = (
            user_df_temp["channels_agg"].str.split(" > ").apply(len)
        )
        user_df_temp["run_date"] = current_date.date()

        # Extract user_guid from journey_id
        user_df_temp['user_guid'] = user_df_temp['journey_id'].str.extract(r'id:(.*)_J:0')[0]

        # Prepare df for merging
        df['conversion_visit_timestamp_date'] = df['conversion_visit_timestamp'].dt.date
        product_arrangement_df = df[['user_guid', 'conversion_visit_timestamp_date', 'product_arrangement_id']].drop_duplicates()

        # Merge user_df_temp with product_arrangement_df
        user_df_temp = user_df_temp.merge(
            product_arrangement_df,
            left_on=['user_guid', 'run_date'],
            right_on=['user_guid', 'conversion_visit_timestamp_date'],
            how='left'
        )

        # Drop 'conversion_visit_timestamp_date' column after merge
        user_df_temp.drop(columns=['conversion_visit_timestamp_date'], inplace=True)

        # Now concatenate user_df_temp into user_df_all_regis_30
        user_df_all_regis_30 = pd.concat(
            [user_df_all_regis_30, user_df_temp], ignore_index=True
        )

        # Proceed with processing markov_transition_matrix and other dataframes as before
        # Markov transition matrix
        markov_transition_matrix = attribution_markov[2].round(3)
        markov_transition_matrix = markov_transition_matrix.rename(
            index=lambda x: x.replace("(inicio)", "(start)"),
            columns=lambda x: x.replace("(inicio)", "(start)"),
        )
        markov_transition_matrix.reset_index(inplace=True)
        markov_transition_matrix = pd.melt(
            markov_transition_matrix,
            id_vars="index",
            var_name="destination",
            value_name="probability",
        )
        markov_transition_matrix.columns = ["source", "destination", "probability"]
        markov_transition_matrix["run_date"] = current_date.date()
        markov_transition_matrix_all_regis_30 = pd.concat(
            [markov_transition_matrix_all_regis_30, markov_transition_matrix],
            ignore_index=True,
        )

        # Removal effects
        removal_effect_matrix = attribution_markov[3].round(3)
        channels = removal_effect_matrix.index
        removal_effect_values = removal_effect_matrix[["removal_effect"]]
        normalized_values = (removal_effect_values / removal_effect_values.sum()) * 100
        normalized_removal_effects = pd.DataFrame(
            normalized_values, index=channels, columns=["removal_effect"]
        )
        normalized_removal_effects["run_date"] = current_date.date()
        normalized_removal_effects["removal_effect_raw"] = (
            removal_effect_values.values.flatten()
        )
        normalized_removal_effects.reset_index(inplace=True)
        normalized_removal_effects.rename(columns={"index": "channel"}, inplace=True)
        normalized_removal_effects_all_regis_30 = pd.concat(
            [normalized_removal_effects_all_regis_30, normalized_removal_effects],
            ignore_index=True,
        )

        # Attribution by channels and models
        attribution_df = attributions.group_by_channels_models
        attribution_df["run_date"] = current_date.date()
        attribution_df.columns = attribution_df.columns.str.replace(
            ".", "_", regex=False
        ).str.replace(" ", "_", regex=False)
        attribution_df_all_regis_30 = pd.concat(
            [attribution_df_all_regis_30, attribution_df], ignore_index=True
        )

        print(f"Processed data for {current_date.strftime('%Y-%m-%d')}")

    except Exception as e:
        print(
            f"An error occurred for the date {current_date.strftime('%Y-%m-%d')}: {e}"
        )
        continue

################################################# Finalize Results #########################################

attribution_df_all_regis_30["conversion_window"] = 30
normalized_removal_effects_all_regis_30["conversion_window"] = 30
markov_transition_matrix_all_regis_30["conversion_window"] = 30
user_df_all_regis_30["conversion_window"] = 30

attribution_df_all_regis_30["conversion_type"] = "registration"
normalized_removal_effects_all_regis_30["conversion_type"] = "registration"
markov_transition_matrix_all_regis_30["conversion_type"] = "registration"
user_df_all_regis_30["conversion_type"] = "registration"